In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from Species import *
from Functions import *

In [2]:
sharks = pd.read_csv('./GSAF5.csv', sep = ',', engine='python') #Usar engine='python', porque si no da error de lectura.

In [3]:
display(sharks.head())

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


## Hipótesis inicial: 
>- De los ataques registrados: ¿Con el paso de los años se ha ido aumentando la **tasa muerte/ataque**? ¿En números totales: cómo han evolucionado los **ataques** y las **muertes**?
>- De los resultados obtenidos sobre la evolución de muertes, intentar encontrar una posible **relación** con la evolución de las **películas sobre tiburones**.
>- Encontrar **especie con mayor tasa muerte/ataque**.
        
#### Para realizar mi análisis deberé tener los siguientes campos limpios:
>- Year, Fatal, Species

#### Para empezar a limpiar, realizo una copia de sharks para hacer la limpieza del DataFrame en un dataframe aparte

In [4]:
sharks_copy = sharks.copy()

In [5]:
print(sharks_copy.columns)

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


In [6]:
display(sharks_copy.isnull().sum()) 

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

#### Podemos ver que las columnas 'Unnamed: 22' y 'Unnamed: 23' vienen con casi todos los registros vacíos:
>- A continuación elimino dichas columnas

In [7]:
sharks_copy.drop(['Unnamed: 22', 'Unnamed: 23'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989


 -  Con el siguiente comando, eliminaré los espacios del principio y del final de los nombres de las columnas

In [8]:
sharks_copy.rename(str.strip, axis='columns', inplace=True)
sharks_copy.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [9]:
display(sharks[(sharks['Case Number']!= sharks['Case Number.1'])|(sharks['Case Number']!= sharks['Case Number.2'])][['Case Number', 'Case Number.1', 'Case Number.2']])

,Case Number,Case Number.1,Case Number.2
4,2016.09.15,2016.09.16,2016.09.15
33,2016.07.14.4,2016.07.14.R,2016.07.14.4
97,2016.01.24.b,2015.01.24.b,2016.01.24.b
116,2015.12.23,2015.11.07,2015.12.23
121,2015.10.28.a,2015.10.28,2015.10.28.a
169,2015.07-10,2015.07.10,2015.07.10
3296,1967.07.05,1967/07.05,1967.07.05
3569,"1962,08.30.b",1962.08.30.b,"1962,08.30.b"
3654,1961.09.02.R,"1961.09,06.R",1961.09.02.R
4177,1952.08.05,1952.08.04,1952.08.05


Comprobamos que únicamente hay 13 registros en los que la info de las columnas es diferente, y se puede ver que difiere en formato (. , /), que ya sustituiría más adelante si fuera necesario:
> - Elimino columnas **'Case Number.1'** y **'Case Number.2'**, puesto que sus valores son los mismos que los de la columna Case Number.

In [10]:
sharks_copy.drop(['Case Number.1', 'Case Number.2'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


A continuación elimino la columna **'pdf'**, puesto que sus valores son los nombres de los archivos, pero no disponemos de ellos.

In [11]:
sharks_copy.drop(['pdf'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,href formula,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,"Orlando Sentinel, 9/19/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,"The Age, 9/18/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,"The Age, 9/16/2016",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [12]:
sharks_copy[['Case Number','Investigator or Source']].head()

,Case Number,Investigator or Source
0,2016.09.18.c,"Orlando Sentinel, 9/19/2016"
1,2016.09.18.b,"Orlando Sentinel, 9/19/2016"
2,2016.09.18.a,"Orlando Sentinel, 9/19/2016"
3,2016.09.17,"The Age, 9/18/2016"
4,2016.09.15,"The Age, 9/16/2016"


Para el análisis que voy a realizar no necesito saber quién ha realizado el estudio ni lo ha publicado. Por lo que procedo a eliminar la columna 'Investigator or Source'

In [13]:
sharks_copy.drop(['Investigator or Source'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,href,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


In [14]:
display(sharks[(sharks['href formula']!= sharks['href'])][['href formula', 'href']].head())
display(sharks[(sharks['href formula']!= sharks['href'])][['href formula', 'href']].shape)

,href formula,href
20,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
27,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
61,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
107,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...
114,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...


(54, 2)

Podemos ver cómo tan solo hay 54 registros en los que difiere la información recogida en las dos columnas 'href' y 'href formula', por lo que voy a **eliminar la columna href**.

In [15]:
sharks_copy.drop(['href'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,original order
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5993
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5992
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5991
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,5990
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,5989


A continuación vemos la estadísticas principales de la columna 'YEAR'

In [16]:
sharks["Year"].describe()

count    5992.000000
mean     1925.204606
std       286.473712
min         0.000000
25%      1942.000000
50%      1975.000000
75%      2003.000000
max      2016.000000
Name: Year, dtype: float64

Hay datos con año 0, empiezo a ver la información que viene en la columna 'date' de los casos con año 0. (sharks_0)

In [17]:
rows_to_delete= sharks_copy[(sharks_copy['Year']==0)]
rows_to_delete['Case Number'].value_counts().head()

ND-1940    1
ND.0039    1
ND.0030    1
ND.0001    1
ND.0048    1
Name: Case Number, dtype: int64

Se puede ver que no viene la información suficiente en la columna 'Case number' para poder obtener los años de estos registros.

In [18]:
print(sharks_copy[sharks_copy["Year"] == 0].shape)

(124, 17)


Tan solo hay 124 registros con Year == 0, por lo que procedo a **eliminar estos registros**.

In [19]:
sharks_copy.drop(rows_to_delete.index, axis=0, inplace = True)
sharks_copy.shape

(5868, 17)

Al importar el archivo functions se han importado funciones definidas **para obtener el año, mes y día del ataque desde la columna 'Case Number'**.
Procedo a aplicar estas funciones para obtener año, mes y día de los ataques.

In [20]:
sharks_copy['Year2']=sharks_copy['Case Number'].apply(year)
sharks_copy['Year2'].value_counts().head()

2015    139
2011    128
2014    125
2013    122
2008    121
Name: Year2, dtype: int64

In [21]:
sharks_copy['Month']=sharks_copy['Case Number'].apply(month)
sharks_copy['Month'].value_counts()

7     634
8     580
9     524
1     500
6     474
12    423
4     410
10    407
3     398
0     398
11    393
5     368
2     359
Name: Month, dtype: int64

Vemos que se han obtenido registros con **mes 0**. En el caso de hacer un análisis por meses, excluiría estos registros de dicho análisis, pero al tener año, decido no eliminar dichos registros.

In [22]:
sharks_copy.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'href formula', 'original order', 'Year2', 'Month'],
      dtype='object')

Compruebo en que difieren los años obtenidos del Case number con los registrados en YEAR, y se ve que sólo hay 5 registros diferentes.

In [23]:
display(sharks_copy[(sharks_copy['Year']!= sharks_copy['Year2'])][['Year', 'Year2']])

,Year,Year2
2449,1969,1989
3662,1971,1961
4312,1848,1948
4983,1922,1923
5043,1919,1900


Intento ver qué ha podido fallar.

In [24]:
sharks_copy.iloc[4312]

Case Number                                            1948.09.17.R
Date                                           Reported 17-Sep-1848
Year                                                           1848
Type                                                     Unprovoked
Country                                                      TURKEY
Area                                                 Adana Province
Location                                                 Yumurtalik
Activity                                                   Swimming
Name                                                     Ali Kaymaz
Sex                                                               M
Age                                                             NaN
Injury                                                        FATAL
Fatal (Y/N)                                                       Y
Time                                                            NaN
Species                                         

Me quedo con la columna original **'YEAR'**, por lo que elimino la columna **'Year2'**.
<br>
Aprovecho para **eliminar** dos columnas que no voy a necesitar en mi análisis: **'Original order'** y **'Name'**.

In [25]:
sharks_copy.drop(['Year2'], axis=1, inplace = True)
sharks_copy.drop(['original order'], axis=1, inplace = True)
sharks_copy.drop(['Name'], axis=1, inplace = True)
sharks_copy.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,Month
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,9
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,9


#### Reordeno columnas para que aparezca año y a continuación mes.

In [26]:
cols=sharks_copy.columns
print(cols)
cols = ['Year','Month', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species', 'href formula', 'Case Number', 'Date']
sharks_copy = sharks_copy[cols]
sharks_copy.head()

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species',
       'href formula', 'Month'],
      dtype='object')


,Year,Month,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,Case Number,Date
0,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,18-Sep-16
1,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,18-Sep-16
2,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,18-Sep-16
3,2016,9,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,17-Sep-16
4,2016,9,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15,16-Sep-16


In [27]:
dict_meses= {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dic'
}

In [28]:
sharks_copy.describe()

,Year,Month
count,5868.000000,5868.000000
mean,1965.887185,6.134799
std,61.730863,3.629566
min,5.000000,0.000000
25%,1945.000000,3.000000
50%,1976.000000,7.000000
75%,2004.000000,9.000000
max,2016.000000,12.000000


Vemos que el percentil 25 se sitúa en 1945. <br>Decido que los datos anteriores a 1949 los voy a eliminar.

In [29]:
rows_to_delete3= sharks_copy[(sharks_copy['Year']<1949)]
sharks_copy.drop(rows_to_delete3.index, axis=0, inplace = True)
sharks_copy.shape

(4302, 16)

In [30]:
sharks_copy.head()

,Year,Month,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,Case Number,Date
0,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,18-Sep-16
1,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,18-Sep-16
2,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,18-Sep-16
3,2016,9,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,17-Sep-16
4,2016,9,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15,16-Sep-16


In [80]:
display(sharks_copy[(sharks_copy['Month']==0)].shape) #Número de registros con mes 0.

(178, 16)

In [32]:
list(set(sharks_copy['Fatal (Y/N)']))

[nan, 'UNKNOWN', ' N', 'N', 'Y']

A continuación quitamos espacios por delante y por detrás de los valores de Fatal

In [ ]:
sharks_copy['Fatal (Y/N)']= sharks_copy['Fatal (Y/N)'].str.strip()

In [35]:
sharks_copy['Fatal (Y/N)'].value_counts()

N          3468
Y           774
UNKNOWN      52
Name: Fatal (Y/N), dtype: int64

Cambio valores null por UNKNOWN al desconocer si murieron finalmente. Afecta a 8 registros.

In [81]:
display(sharks_copy['Fatal (Y/N)'].value_counts())
sharks_copy['Fatal (Y/N)'].fillna('UNKNOWN', inplace=True)

N          3468
Y           774
UNKNOWN      60
Name: Fatal (Y/N), dtype: int64

In [37]:
display(sharks_copy.head())

,Year,Month,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,Case Number,Date
0,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,16,Minor injury to thigh,N,13h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,18-Sep-16
1,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,36,Lacerations to hands,N,11h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,18-Sep-16
2,2016,9,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,43,Lacerations to lower leg,N,10h43,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,18-Sep-16
3,2016,9,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,NaN,Struck by fin on chest & leg,N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,17-Sep-16
4,2016,9,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,NaN,No injury: Knocked off board by shark,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15,16-Sep-16


A continuación comprobamos cuántos valores null tenemos en nuestro DataFrame:

In [38]:
sharks_copy.isnull().sum()

Year               0
Month              0
Type               0
Country           14
Area             206
Location         261
Activity         318
Sex              397
Age             1410
Injury            16
Fatal (Y/N)        0
Time            1857
Species         1673
href formula       1
Case Number        0
Date               0
dtype: int64

Y sus tipos de datos:

In [39]:
sharks_copy.dtypes

Year             int64
Month            int64
Type            object
Country         object
Area            object
Location        object
Activity        object
Sex             object
Age             object
Injury          object
Fatal (Y/N)     object
Time            object
Species         object
href formula    object
Case Number     object
Date            object
dtype: object

Procedo a **sustituir** todos los valores **null** por un valor **'UNKNOWN'**, ya que se desconoce la información de los campos para esos registros.

In [40]:
sharks_copy['Country'].fillna('UNKNOWN', inplace=True)
sharks_copy['Area'].fillna('UNKNOWN', inplace=True)
sharks_copy['Location'].fillna('UNKNOWN', inplace=True)
sharks_copy['Activity'].fillna('UNKNOWN', inplace=True)
sharks_copy['Age'].fillna('UNKNOWN', inplace=True)
sharks_copy['Injury'].fillna('UNKNOWN', inplace=True)
sharks_copy['Time'].fillna('UNKNOWN', inplace=True)
sharks_copy['Species'].fillna('UNKNOWN', inplace=True)
sharks_copy['href formula'].fillna('UNKNOWN', inplace=True)
sharks_copy['Sex'].fillna('UNKNOWN', inplace=True)

Procedo a eliminar espacios en blanco que hay por delante y por detrás en los valores registrados en la columna 'SEX'

In [41]:
sharks_copy['Sex']= sharks_copy['Sex'].str.strip()
sharks_copy['Sex'].value_counts()

M          3412
F           492
UNKNOWN     397
lli           1
Name: Sex, dtype: int64

Y remplazo el valor 'lli' por 'UNKNOWN'

In [42]:
sharks_copy['Sex'].replace('lli','UNKNOWN', inplace=True)
sharks_copy['Sex'].value_counts()

M          3412
F           492
UNKNOWN     398
Name: Sex, dtype: int64

Comprobamos que nuestro DataFrame ya no tiene valores null

In [43]:
sharks_copy.isnull().sum()

Year            0
Month           0
Type            0
Country         0
Area            0
Location        0
Activity        0
Sex             0
Age             0
Injury          0
Fatal (Y/N)     0
Time            0
Species         0
href formula    0
Case Number     0
Date            0
dtype: int64

In [83]:
sharks_copy.columns

Index(['Year', 'Month', 'Type', 'Country', 'Area', 'Location', 'Activity',
       'Sex', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species',
       'href formula', 'Case Number', 'Date'],
      dtype='object')

In [82]:
sharks_copy['Species'].shape

(4302,)

Al importar el módulo **Species**, he importado la función para limpiar los tipos de especies de tiburones registradas en nuestro DataFrame y así normalizarlas **** normal_species

In [49]:
sharks_copy['Species']=sharks_copy['Species'].apply(normal_species)

In [52]:
sharks_copy['Species'].fillna('Unknown', inplace=True)
sharks_copy.isnull().sum()

Year            0
Month           0
Type            0
Country         0
Area            0
Location        0
Activity        0
Sex             0
Age             0
Injury          0
Fatal (Y/N)     0
Time            0
Species         0
href formula    0
Case Number     0
Date            0
dtype: int64

In [50]:
display(sharks_copy['Species'].value_counts().sum())

4271

In [51]:
sharks_copy['Species'].value_counts()

Unknown                            1688
Other                               809
White Shark                         536
Tiger Shark                         204
Bull Shark                          191
Sand Tiger Shark                    162
Shark involvement not confirmed     124
Blacktip Shark                       89
Bronze Whaler Shark                  72
Nurse Shark                          49
Shortfin Mako Shark                  48
Great Hammerhead Shark               43
Wobbegong Shark                      43
Blue Shark                           40
Lemon Shark                          28
Oceanic Whitetip Shark               27
Gray Reef Shark                      27
Spinner Shark                        20
Broadnose Sevengill Shark            16
Blacktip Reef Shark                  14
Dusky Shark                          11
Carpet Shark                          8
Dogfish Shark                         7
Sandbar Shark                         6
Galapagos Shark                       4


#### He obtenido una base de datos de IMBD que contiene la filmografía existente relacionada con tiburones.

In [96]:
movie_sharks = pd.read_csv('./ALL shark movies.csv', sep = ',', engine='python')

In [97]:
movie_sharks = movie_sharks [['Position','Title', 'Year', 'Title Type']]

In [121]:
movie_sharks['Title Type'].value_counts()

movie    56
Name: Title Type, dtype: int64

Procedo a eliminar los registros anteriores a 1949.

In [98]:
rows_to_delete= movie_sharks[(movie_sharks['Year']<1949)|(movie_sharks['Year'].isnull())|(movie_sharks['Year']>2016)]

In [99]:
movie_sharks.drop(rows_to_delete.index, axis=0, inplace = True)

Eliminamos de la base de datos toda la filmografía que no sea 'movie', puesto que lo quiero relacionar únicamente con las películas de cine.

In [100]:
rows_to_delete2= movie_sharks[(movie_sharks['Title Type']!= 'movie')]

In [101]:
movie_sharks.drop(rows_to_delete2.index, axis=0, inplace = True)

In [102]:
movie_sharks.head()

,Position,Title,Year,Title Type
12,13,Mega Shark vs. Mecha Shark,2014.0,movie
13,14,Mega Shark vs. Kolossus,2015.0,movie
20,21,Ozark Sharks,2016.0,movie
23,24,Sand Sharks,2012.0,movie
26,27,Ghost Shark 2: Urban Jaws,2015.0,movie


#### Voy a crear un nuevo data frame en el que se va a incluir el número de películas sobre tiburones por años en la serie estudiada.

In [103]:
movies_byyear=pd.DataFrame()

In [104]:
year = [i for i in range(1949,2017)]

In [105]:
movies_byyear['Year']= year
movies_byyear.head()

,Year
0,1949
1,1950
2,1951
3,1952
4,1953


In [106]:
df1=pd.DataFrame()
df1['Movies']=movie_sharks.groupby(['Year']).count()['Title']
df1 = df1.reset_index().rename_axis(None).rename_axis(None, axis=1)
df1['Year']= df1['Year'].astype(int)
df1.dtypes

Year      int64
Movies    int64
dtype: object

In [107]:
movies_byyear= movies_byyear.merge(df1,how='left', on='Year')

movies_byyear.head()

,Year,Movies
0,1949,NaN
1,1950,1.0
2,1951,NaN
3,1952,NaN
4,1953,NaN


In [108]:
movies_byyear['Movies'].fillna(0, inplace=True)

In [115]:
movies_byyear.set_index('Year', inplace=True)

In [117]:
movies_byyear = movies_byyear.reset_index().rename_axis(None).rename_axis(None, axis=1)

In [118]:
movies_byyear.head()

,Year,Movies
0,1949,0.0
1,1950,1.0
2,1951,0.0
3,1952,0.0
4,1953,0.0


In [122]:
sharks_copy[(sharks_copy['Fatal (Y/N)']=='Y')& (sharks_copy['Year']==1969)]

,Year,Month,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal (Y/N),Time,Species,href formula,Case Number,Date


In [75]:
sharks_copy.to_csv('Data_clean.csv', index=False)

In [123]:
movies_byyear.to_csv('Shark movies.csv', index=False)

In [77]:
#sharks_0['Date'] = sharks_0['Date'].str.replace('?', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace(r'\w+efore ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Beforer ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace(r'Between', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Circa ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Early ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace(' or ', ',')
#sharks_0['Date'] = sharks_0['Date'].str.replace('-', ',')
#sharks_0['Date'] = sharks_0['Date'].str.replace('&', ',')
#sharks_0['Date'] = sharks_0['Date'].str.replace('Ca. ', '-')
#sharks_0['Date'] = sharks_0['Date'].str.replace(' B.C.', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('.B.C..', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('World War II', '1945')
#sharks_0['Date'] = sharks_0['Date'].str.replace('A few years ', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('s', '')
#sharks_0['Date'] = sharks_0['Date'].str.replace('No date, ', '')
#sharks_0['Date']

In [78]:
#list(set(sharks_0['Date']))

In [79]:
#sharks_0['Case Number']